In [1]:
BROKER = "mqtt.eclipseprojects.io"
TOPIC = "/gw01/duese03"

#BROKER = "broker.hivemq.com"
#TOPIC = "/gw01/duese02"

import paho.mqtt.client as mqtt
import numpy as np
#from numpy import zeros
import bqplot.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

In [2]:
from bqplot import (
    LinearScale,
    Axis,
    Figure,
    OrdinalScale,
    LinearScale,
    Bars,
    Lines,
    Scatter,
)

In [3]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('HERE THE DECODED MQTT MESSAGES WILL BE SHOWN!\n')
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [7]:
# Create an initial empty line plot
# x_data = np.array([])  # Initialize with empty arrays
# z_data = np.array([])
# t_data = np.array([])

N = 20

x_data = np.array(np.arange(0,N)*5.)  # Initialize with empty arrays
z_data = np.array(np.zeros(N))  # Initialize with empty arrays
t_data = np.array(np.zeros(N))  # Initialize with empty arrays


xsc = LinearScale(min = -10, max = 410)
ysc = LinearScale(min = 0, max = 2)

# 2. Create the axes for x and y
xax = Axis(scale = xsc, label = "X")
yax = Axis(scale = ysc, orientation = "vertical", label = "Z")

# 3. Create a Lines mark by passing in the scales
# note that Lines object is stored in `line` which can be used later to update the plot
line = Lines(x = x_data, y = z_data, scales={"x": xsc, "y": ysc})

# 4. Create a Figure object by assembling marks and axes
fig = Figure(marks=[line], axes=[xax, yax], title="Simple Line Chart")

# 5. Render the figure using display or just as is
#display(fig)

i = 0
t = x = z = 0
payload = ""

def on_message(client, userdata, message):
    global i
    global payload
    global t,x,z
    global x_data
    global z_data
    global t_data
    global line
    
    # Parse the received message
    payload = message.payload.decode('utf-8')
    t, x, z = map(float, payload.strip('()').split(','))
    s = f"{i:4d}  {t:12.2f}  {x:6.2f}  {z:6.2f}"
    i+=1

#    print(s)
#    out.append_stdout(s+"\n")

    # Append the new data point to the existing data

    # x_data = np.append(x_data, x)
    # z_data = np.append(z_data, z)
    # t_data = np.append(t_data, t)

    t_data = np.roll(t_data, -1); t_data[-1] = t    
    x_data = np.roll(x_data, -1); x_data[-1] = x
    z_data = np.roll(z_data, -1); z_data[-1] = z
    
    # # Update the line plot with new data
    line.x = x_data
    line.y = z_data

    # # Update the text widget with the latest 't' value
    text_widget.text = [f"Latest t: {t_data[-1]-t_data[0]:.1f} s"]

    # Refresh the plot
#    clear_output(wait=True)
#    display(fig)

# Initialize the MQTT client#
client = mqtt.Client()
client.on_message = on_message
#clienbroker.hivemq.comprojects.io", 1883, 60)
#client.subscribe("/gw01/duese01")
client.connect(BROKER, 1883, 60)
client.subscribe(TOPIC)

while (client.is_connected == False): pass

out.append_stdout("MQTT Client connected.\n")

# Start the MQTT client loop in the background
client.loop_start()

# Display the figure
display(fig)


Figure(axes=[Axis(label='X', scale=LinearScale(max=410.0, min=-10.0)), Axis(label='Z', orientation='vertical',…

In [5]:
len(z_data)

20

In [6]:
x_data

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.,
       65., 70., 75., 80., 85., 90., 95.])